In [7]:
import numpy as np
import pyvista as pv
import torch
from numba import jit, prange
from scipy.spatial import cKDTree

In [2]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')

In [3]:
# cyan = torch.Tensor(np.logical_and(etching[:50, :, :,1]==10, etching[:50, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:50, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:50, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:50, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61109/index.html?ui=P_0x18930c2a3c0_0&reconnect=auto" class="pyvis…

In [4]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [ ]:
    # def scanZ(self, film): # fast scanZ
    #     film = torch.Tensor(film)
    #     xshape, yshape, zshape = film.shape
    #     self.zshape = zshape
    #     # 初始化一个全零的表面稀疏张量
    #     surface_sparse = torch.zeros((xshape, yshape, zshape))
        
    #     # 获取当前平面与前后平面的布尔索引
    #     current_plane = film != 0

    #     # 获取周围邻居的布尔索引
    #     neighbors = torch.zeros_like(film, dtype=torch.bool)
        
    #     neighbors[1:, :, :] |= film[:-1, :, :] == 0  # 上面
    #     neighbors[:-1, :, :] |= film[1:, :, :] == 0  # 下面
    #     neighbors[:, 1:, :] |= film[:, :-1, :] == 0  # 左边
    #     neighbors[:, :-1, :] |= film[:, 1:, :] == 0  # 右边
    #     neighbors[:, :, 1:] |= film[:, :, :-1] == 0  # 前面
    #     neighbors[:, :, :-1] |= film[:, :, 1:] == 0  # 后面
        
    #     # 获取满足条件的索引
    #     condition = current_plane & neighbors
        
    #     # 更新表面稀疏张量
    #     surface_sparse[condition] = 1
        
    #     return surface_sparse.to_sparse()

In [29]:
def getFloat_pointcloud(film):  # fast scanZ
    sumFilm = np.sum(film, axis=-1)
    sumFilm = torch.Tensor(sumFilm)
    film = torch.Tensor(film)
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0
    # 初始化一个全零的表面稀疏张量
    surface_sparse = torch.zeros_like(sumFilm)
    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = torch.zeros_like(filmC, dtype=torch.bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    surface_sparse[condition] = 1

    points = surface_sparse.to_sparse().indices().T
    return points

In [ ]:
def removeFloat(film):  # fast scanZ
    filmC = film[:,:,:,0]
    # 获取当前平面的非零元素布尔索引
    current_plane = film[:,:,:,1] != 0

    # 创建一个全是False的布尔数组来存储邻居的检查结果
    neighbors = np.zeros_like(filmC, dtype=bool)

    # 检查各个方向的邻居是否为零
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面的邻居不为0
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面的邻居不为0
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边的邻居不为0
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边的邻居不为0
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面的邻居不为0
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面的邻居不为0

    # 孤立单元格的条件是当前平面元素不为0且所有方向的邻居都为0
    condition = current_plane & ~neighbors

    # 将孤立的单元格设为0
    film[condition, :] = 0

    return film

In [96]:
def scanZ(film): # fast scanZ
    xshape, yshape, zshape, layer = film.shape
    filmC = film[:,:,:,0]
    # 初始化一个全零的表面稀疏张量
    surface_sparse = np.zeros((xshape, yshape, zshape))
    
    # 获取当前平面与前后平面的布尔索引
    current_plane = np.sum(film, axis=-1) == 0

    # 获取周围邻居的布尔索引
    neighbors = np.zeros_like(filmC, dtype=np.bool_)
    
    neighbors[1:, :, :] |= filmC[:-1, :, :] != 0  # 上面
    neighbors[:-1, :, :] |= filmC[1:, :, :] != 0  # 下面
    neighbors[:, 1:, :] |= filmC[:, :-1, :] != 0  # 左边
    neighbors[:, :-1, :] |= filmC[:, 1:, :] != 0  # 右边
    neighbors[:, :, 1:] |= filmC[:, :, :-1] != 0  # 前面
    neighbors[:, :, :-1] |= filmC[:, :, 1:] != 0  # 后面
    
    # 获取满足条件的索引
    condition = current_plane & neighbors
    
    # 更新表面稀疏张量
    surface_sparse[condition] = 1
    
    return surface_sparse

In [97]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')
plane_point = scanZ(etching)
# print(plane_point.shape)


plane_tree = cKDTree(np.argwhere(plane_point == 1))
Float_point = getFloat_pointcloud(etching)
print(Float_point.shape)
# indice_all = np.zeros_like(pos.shape[0], dtype=np.bool_)
dd, ii = plane_tree.query(Float_point, k=1, workers=1)
surface_indice = np.argwhere(plane_point)
i1 = surface_indice[ii][:,0] #[particle, order, xyz]
j1 = surface_indice[ii][:,1]
k1 = surface_indice[ii][:,2]
print(ii.shape)
print('ii', ii)
print(surface_indice.shape)
# print(np.max(ii))
print(Float_point.shape)
print(etching.shape)
Float_point = Float_point.numpy()
print('Float_point',Float_point.shape)
print(surface_indice[ii].shape)
print(etching[i1, j1, k1, 1].shape)
# etching[i1, j1, k1, 1] += Float_point[:, 1]
# print(ii)
# etching[]

torch.Size([6207, 3])
(6207,)
ii [2069 2069 2069 ... 4423 4423 4423]
(4438, 3)
torch.Size([6207, 3])
(15, 300, 690, 3)
Float_point (6207, 3)
(6207, 3)
(6207,)


In [110]:
def depoFloat(film):
    plane_point = scanZ(film)

    plane_tree = cKDTree(np.argwhere(plane_point == 1))
    Float_point = getFloat_pointcloud(film)

    dd, ii = plane_tree.query(Float_point, k=1, workers=1)
    surface_indice = np.argwhere(plane_point)
    i1 = surface_indice[ii][:,0] #[particle, order, xyz]
    j1 = surface_indice[ii][:,1]
    k1 = surface_indice[ii][:,2]

    Float_point = Float_point.numpy()
    film[i1, j1, k1, 1] += 20
    film[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],1] = 30

    return film

In [111]:
etching = np.load('./bosch_data_1012_ratio08_trench_condition5_300wide/bosch_sf_step_12_sf.npy')
etching = depoFloat(etching)

In [112]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]==10, etching[:, :, :,0]==0)).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==20).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,1]==30).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61109/index.html?ui=P_0x189c0ba8b90_18&reconnect=auto" class="pyvi…

Exception in callback _ProactorBasePipeTransport._call_connection_lost(None)
handle: <Handle _ProactorBasePipeTransport._call_connection_lost(None)>
Traceback (most recent call last):
  File "c:\Users\urcs049\AppData\Local\anaconda3\Lib\asyncio\events.py", line 88, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Users\urcs049\AppData\Local\anaconda3\Lib\asyncio\proactor_events.py", line 165, in _call_connection_lost
    self._sock.shutdown(socket.SHUT_RDWR)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host


In [5]:
etching = removeFloat(etching)

In [80]:
etchingB = np.copy(etching)

In [98]:
etching[i1, j1, k1, 1] += 10

In [101]:
etching[Float_point[:, 0],Float_point[:, 1],Float_point[:, 2],1] = 0

In [87]:
np.allclose(etching, etchingB)

False

In [88]:
etching.shape

(15, 300, 690, 3)

In [89]:
etchingB.shape

(15, 300, 690, 3)

In [86]:
np.allclose(etching[:, 1], etchingB[:, 1])

True

In [84]:
np.allclose(etching[:, 0], etchingB[:, 0])

True

In [85]:
np.allclose(etching[:, 2], etchingB[:, 2])

True

In [102]:
# cyan = torch.Tensor(np.logical_and(etching[:, :, :,1]!=10, etching[:, :, :,1]!=0)).to_sparse()
# cyan = torch.Tensor(etching[:, 1] != etchingB[:, 1]).to_sparse()
cyan = torch.Tensor(etching[:, :, :,1]==10).to_sparse()
cyan = cyan.indices().numpy().T

gray = torch.Tensor(etching[:, :, :,0]!=0).to_sparse()
gray = gray.indices().numpy().T

mask = torch.Tensor(etching[:, :, :,2]!=0).to_sparse()
mask = mask.indices().numpy().T

maskmesh = pv.PolyData(mask)
maskmesh["radius"] = np.ones(mask.shape[0])*0.5

depomesh = pv.PolyData(cyan)
depomesh["radius"] = np.ones(cyan.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(gray)
submesh["radius"] = np.ones(gray.shape[0])*0.5

# Progress bar is a new feature on master branch
maskglyphed = maskmesh.glyph(scale="radius", geom=geom) # progress_bar=True)
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(maskglyphed, color='green')
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:61109/index.html?ui=P_0x1899e8f61e0_15&reconnect=auto" class="pyvi…